In [135]:
import src.data_mgmt.RecSysData as rsd
from torch.utils.data import DataLoader
import torch
import pandas as pd
import numpy as np
from src.data_mgmt.BaseDataClass import BaseDataClass as bdc
from tqdm import tqdm_notebook as tqdm
from src.data_mgmt.ValidationBaseDataClass import ValidationDataClass as vdc
from sklearn.metrics import mean_squared_error as mse

In [5]:
#def _loadUpDf(pathname, filename, extensions):
           
df = bdc._loadUpDf('/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/','train','.json.gz')

Loading JSON into Dataframe: 200000it [00:15, 13330.90it/s]


In [7]:
df.columns

Index(['reviewHash', 'reviewTime', 'reviewText', 'nHelpful', 'outOf',
       'reviewerID', 'reviewHash', 'categories', 'unixReviewTime', 'itemID',
       'rating', 'summary', 'categoryID', 'helpfulPerc',
       'summaryCharacterLength', 'reviewTextCharacterLength', 'parentCategory',
       'cat1', 'cat2', 'cat3', 'cat1_child', 'cat2_parent', 'cat2_child'],
      dtype='object')

In [8]:
df.head()

,reviewHash,reviewTime,reviewText,nHelpful,outOf,reviewerID,reviewHash,categories,unixReviewTime,itemID,...,helpfulPerc,summaryCharacterLength,reviewTextCharacterLength,parentCategory,cat1,cat2,cat3,cat1_child,cat2_parent,cat2_child
0,R798569390,2013-09-26,The model in this picture has them rolled up a...,0,0,U490934656,R798569390,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",1380153600,I402344648,...,NaN,12,276,"Clothing, Shoes & Jewelry",Women,,,Women,"Clothing, Shoes & Jewelry",Leggings
1,R436443063,2013-02-07,"I love the look of this bra, it is what I want...",0,0,U714157797,R436443063,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1360195200,I697650540,...,NaN,29,269,"Clothing, Shoes & Jewelry",Women,Clothing,"Lingerie, Sleep & Lounge",Everyday Bras,"Clothing, Shoes & Jewelry",Everyday Bras
2,R103439446,2014-03-16,I am not comfortable with wearing my wedding b...,0,0,U507366950,R103439446,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",1394928000,I464613034,...,NaN,28,1073,"Clothing, Shoes & Jewelry",Wedding Party Gifts,Gifts for the Groomsmen,,Gifts for the Groomsmen,"Clothing, Shoes & Jewelry",Women
3,R486351639,2014-03-10,Like the look of this top and really looks cut...,0,0,U307862152,R486351639,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1394409600,I559560885,...,NaN,32,464,"Clothing, Shoes & Jewelry",Women,Clothing,Swimsuits & Cover Ups,Cover-Ups,,
4,R508664275,2013-07-30,I'm quite small and the XS fits me like a regu...,1,1,U742726598,R508664275,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",1375142400,I476005312,...,1.0,11,424,"Clothing, Shoes & Jewelry",Women,Plus-Size,Tops & Tees,Knits & Tees,"Clothing, Shoes & Jewelry",T-Shirts


In [21]:
ppath='/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/'

train_set = rsd.RecSysData(ppath)

In [22]:
train_set.df_data.head()

,reviewHash,reviewerID,unixReviewTime,itemID,rating,uid,pid
0,R798569390,U490934656,1380153600,I402344648,4.0,0,0
1,R436443063,U714157797,1360195200,I697650540,4.0,1,1
2,R103439446,U507366950,1394928000,I464613034,5.0,2,2
3,R486351639,U307862152,1394409600,I559560885,2.0,3,3
4,R508664275,U742726598,1375142400,I476005312,5.0,4,4


In [32]:
b_size = 4

train_loader = DataLoader(tt, batch_size=b_size, shuffle=True)

Questions: All users in the dataset purchased the item already, how do we want to determine in test set if item will be purchased or not

Baseline looks at number of times and item is purchased and if item is in the top 50% of purchases, predict that test user will purchase.


Ideas: Top highly rated items? 

User Rec Sys - > if we predict higher than 3.5, user will purchase?

In [60]:
#creating test set based on timing of ratings
# train/test set division chosen so test set contains about 20% of data (19.878%)
df['review_date_rank'] = df.groupby('reviewerID')['reviewTime'].rank(ascending=False)
testset = df[df['review_date_rank'] < 2].copy()
trainset = df[df['review_date_rank'] >= 2].copy()

In [90]:
df.iloc[2,:]

reviewHash                                                          R103439446
reviewTime                                                 2014-03-16 00:00:00
reviewText                   I am not comfortable with wearing my wedding b...
nHelpful                                                                     0
outOf                                                                        0
reviewerID                                                          U507366950
reviewHash                                                          R103439446
categories                   [[Clothing, Shoes & Jewelry, Wedding Party Gif...
unixReviewTime                                                      1394928000
itemID                                                              I464613034
rating                                                                     5.0
summary                                           Great Alternative for Nurses
categoryID                                          

In [109]:
df.iloc[2,5]

'U507366950'

In [121]:
total_sample = len(df)
train_sample = len(trainset)
test_sample = len(testset)

train_user_diction = {}   # store data using userID as keys
train_item_diction = {}   # store data using itemID as keys
test_user_diction = {}   # store data using userID as keys
test_item_diction = {}   # store data using itemID as keys

for index in df.index:
    if index in trainset.index:
        if df.iloc[index, 9] not in train_item_diction:
            train_item_diction[df.iloc[index, 9]] = [[df.iloc[index,5], df.iloc[index,10]]]
        else:
            train_item_diction[df.iloc[index, 9]].append([df.iloc[index,5], df.iloc[index,10]])
        if df.iloc[index, 5] not in train_item_diction:
            train_user_diction[df.iloc[index, 5]] = [[df.iloc[index,9], df.iloc[index,10]]]
        else:
            train_user_diction[df.iloc[index, 5]].append([df.iloc[index,9], df.iloc[index,10]])
    else:
        if df.iloc[index, 9] not in test_item_diction:
            test_item_diction[df.iloc[index, 9]] = [[df.iloc[index,5], df.iloc[index,10]]]
        else:
            test_item_diction[df.iloc[index, 9]].append([df.iloc[index,5], df.iloc[index,10]])
        if df.iloc[index, 5] not in train_item_diction:
            test_user_diction[df.iloc[index, 5]] = [[df.iloc[index,9], df.iloc[index,10]]]
        else:
            test_user_diction[df.iloc[index, 5]].append([df.iloc[index,9], df.iloc[index,10]])

In [127]:
traindata = np.zeros((train_sample, 3), dtype=object)
testdata = np.zeros((test_sample, 3), dtype=object) # store entire data as an array
trainratings = np.zeros(train_sample)   # store all ratings
testratings = np.zeros(test_sample)

for i in range(train_sample):
    traindata[i,0] = trainset.iloc[i,5]
    traindata[i,1] = trainset.iloc[i,9]
    traindata[i,2] = trainset.iloc[i,10]
    trainratings[i] = trainset.iloc[i,10]
    
for i in range(test_sample):
    testdata[i,0] = testset.iloc[i,5]
    testdata[i,1] = testset.iloc[i,9]
    testdata[i,2] = testset.iloc[i,10]
    testratings[i] = testset.iloc[i,10]

In [178]:
alpha_4 = 0
beta_u = {}
beta_i = {}
delta = 1
epoch = 0
while delta >= 1:
    local_alpha = 0
    local_beta_u = {}
    local_beta_i = {}
    
    for user, item, rating in traindata:  # we first update alpha
        if user not in beta_u:
            if item not in beta_i:
                local_alpha += int(rating)/train_sample
            else:
                local_alpha += (int(rating) - beta_i[item])/train_sample
        else:
            if item not in beta_i:
                local_alpha += (int(rating) - beta_u[user])/train_sample
            else:
                local_alpha += (int(rating) - beta_u[user] - beta_i[item])/train_sample
    delta = abs(alpha_4 - local_alpha)
    alpha_4 = local_alpha
    
    for user, item, rating in traindata:  # we first update beta_u
        if item not in beta_i:
            if user not in local_beta_u:
                local_beta_u[user] = (int(rating) - alpha_4)/len(train_user_diction[user])
            else:
                local_beta_u[user] += (int(rating) - alpha_4)/len(train_user_diction[user])
        else:
            if user not in local_beta_u:
                local_beta_u[user] = (int(rating) - alpha_4 - beta_i[item])/len(train_user_diction[user])
            else:
                local_beta_u[user] += (int(rating) - alpha_4 - beta_i[item])/len(train_user_diction[user])
    if epoch !=0 :
        delta = max(delta, abs(list(local_beta_u.values())[0] - list(beta_u.values())[0]))
    beta_u = local_beta_u    
    
    for user, item, rating in traindata:  # we first update beta_i
        if item not in local_beta_i:
            local_beta_i[item] = (int(rating) - alpha_4 - beta_u[user])/len(train_item_diction[item])
        else:
            local_beta_i[item] += (int(rating) - alpha_4 - beta_u[user])/len(train_item_diction[item])
    
    if epoch != 0:
        delta = max(delta, abs(list(local_beta_i.values())[0] - list(beta_i.values())[0]))
    
    beta_i = local_beta_i

    epoch += 1

In [368]:
train_prediction = np.zeros(train_sample)
test_prediction = np.zeros(test_sample)
for i in range(train_sample):
    user = traindata[i,0]
    item = traindata[i,1]
    rating = traindata[i,2]
    train_prediction[i] = alpha_4 + beta_u[user] + beta_i[item]

for i in range(test_sample):
    user = testdata[i,0]
    item = testdata[i,1]
    rating = testdata[i,2]
    if user in beta_u:
        if item in beta_i:
            test_prediction[i] = alpha_4 + beta_u[user] + beta_i[item]
        else:
            test_prediction[i] = alpha_4 + beta_u[user]
    else:
        if item in beta_i:
            test_prediction[i] = alpha_4 + beta_i[item]
        else:
            test_prediction[i] = alpha_4
            
        
train_error = mse(trainratings, train_prediction)
test_error = mse(testratings, test_prediction)
print('train mse', train_error)
print('train rmse', np.sqrt(train_error))
print('testing mse', test_error)
print('testing rmse', np.sqrt(test_error))

train mse 832.1479601613062
train rmse 28.846974887521675
testing mse 197.82125123339185
testing rmse 14.064894284472665


In [538]:
#concatenate predictions and datasets
train_data_new = pd.DataFrame(traindata, columns = ["reviewerID", "itemID", "rating"])
train_data_new['predicted_ratings'] = train_prediction

test_data_new = pd.DataFrame(testdata, columns = ["reviewerID", "itemID", "rating"])
test_data_new['predicted_ratings'] = test_prediction

df_new = pd.concat([train_data_new, test_data_new])

In [469]:
df_new

,reviewerID,itemID,rating,predicted_ratings
0,U490934656,I402344648,4.0,4.536653
1,U714157797,I697650540,4.0,16.282194
2,U307862152,I559560885,2.0,2.057537
3,U659368900,I342631873,4.0,6.087586
4,U361187730,I773829721,5.0,3.077724
...,...,...,...,...
39751,U960733769,I929752512,5.0,3.241847
39752,U922232419,I484262724,5.0,3.156143
39753,U493479353,I406044324,4.0,11.817335
39754,U118520138,I857075665,5.0,4.009850


In [424]:
actualtestset = pd.read_csv('/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/pairs_Purchase.txt')
actualtestset[['reviewerID','itemID']] = actualtestset['reviewerID-itemID'].str.split("-",expand=True)
actualtestset['rating'] = 0

In [500]:
item_set = []
for i in df_new.iloc[:,1]:
    item_set.append(i)
    
user_set = []
for u in df_new.iloc[:,0]:
    user_set.append(u)
    
user_item = {}
for index in df_new.index:
    user = df_new.iloc[index,0]
    item = df_new.iloc[index, 1]
    if user not in user_item:
        user_item[user] = [item]
    else: 
        user_item[user].append(item)

item_user = {}
for index in df_new.index:
    user = df_new.iloc[index, 0]
    item = df_new.iloc[index, 1]
    if item not in item_user:
        item_user[item] = [user]
    else: 
        item_user[item].append([user])

In [486]:
#actualtestset_len = len(actualtestset)
#predicted_ratings_test = np.zeros(actualtestset_len)

#for i in range(actualtestset_len):
#    user = actualtestset.iloc[i,2]
#    item = actualtestset.iloc[i,3]
#    if user in df_new["itemID"]:
#        if item in user_item[user]:
 #           predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["reviewerID"] == user) & (df_new["itemID"] == item)])
 #       else:
 #           predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["reviewerID"] == user)])
 #   else:
 #       if item in df_new["itemID"]:
 #           predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["itemID"] == item)])
  #      else:
   #         predicted_ratings_test[i] = alpha_4

In [502]:
actualtestset_len = len(actualtestset)
predicted_ratings_test = np.zeros(actualtestset_len)

for i in range(actualtestset_len):
    user = actualtestset.iloc[i,2]
    item = actualtestset.iloc[i,3]
    if user in user_item:
        if item in user_item[user]:
            predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["reviewerID"] == user) & (df_new["itemID"] == item)])
        else:
            predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["reviewerID"] == user)])
    else:
        if item in item_user:
            predicted_ratings_test[i] = np.mean(df_new['predicted_ratings'][(df_new["itemID"] == item)])
        else:
            predicted_ratings_test[i] = alpha_4

In [503]:
predicted_ratings_test

array([ 5.89345913,  3.95885961, -2.87273398, ...,  8.72687794,
        0.74122944, -4.31493949])

In [504]:
actualtestset['rating'] = predicted_ratings_test
actualtestset

,reviewerID-itemID,prediction,reviewerID,itemID,rating
0,U938994110-I529819131,NaN,U938994110,I529819131,5.893459
1,U181459539-I863471064,NaN,U181459539,I863471064,3.958860
2,U941668816-I684585522,NaN,U941668816,I684585522,-2.872734
3,U768449391-I782253949,NaN,U768449391,I782253949,12.446827
4,U640450168-I232683472,NaN,U640450168,I232683472,2.121477
...,...,...,...,...,...
27995,U337041888-I763827121,NaN,U337041888,I763827121,9.794353
27996,U457455307-I242828364,NaN,U457455307,I242828364,-2.097524
27997,U052546714-I111529174,NaN,U052546714,I111529174,8.726878
27998,U566804667-I857242737,NaN,U566804667,I857242737,0.741229


In [528]:
for i in range(actualtestset_len):
    if actualtestset.iloc[i,4] >= 3.5:
        actualtestset.iloc[i,1] = 1
    else:
        actualtestset.iloc[i,1] = 0

In [529]:
actualtestset

,reviewerID-itemID,prediction,reviewerID,itemID,rating
0,U938994110-I529819131,1.0,U938994110,I529819131,5.893459
1,U181459539-I863471064,1.0,U181459539,I863471064,3.958860
2,U941668816-I684585522,0.0,U941668816,I684585522,-2.872734
3,U768449391-I782253949,1.0,U768449391,I782253949,12.446827
4,U640450168-I232683472,0.0,U640450168,I232683472,2.121477
...,...,...,...,...,...
27995,U337041888-I763827121,1.0,U337041888,I763827121,9.794353
27996,U457455307-I242828364,0.0,U457455307,I242828364,-2.097524
27997,U052546714-I111529174,1.0,U052546714,I111529174,8.726878
27998,U566804667-I857242737,0.0,U566804667,I857242737,0.741229


In [527]:
actualtestset[['reviewerID-itemID','prediction']].to_csv('Andrea-Purchase-2021-12-11-v4.csv')

In [542]:
train_prediction

array([  4.53665291,  16.28219426,   2.05753708, ..., -11.85431781,
         0.16792036,   1.79873885])

In [543]:
trainratings

array([4., 4., 2., ..., 1., 4., 5.])

In [544]:
traindata[1,]

array(['U714157797', 'I697650540', 4.0], dtype=object)

In [549]:
beta_u['U714157797'] + beta_i['I697650540'] + alpha_4

16.282194264240943